In [2]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

df = pd.read_csv(
    ROOT / "outputs" / "tables" / "phase3_engineered.csv.gz",
    compression="gzip",
    low_memory=False,
)

print("Loaded:", df.shape)
print(df.columns.tolist())
conf_map = {"Low": 1, "Medium": 2, "High": 3, "Unknown": 1}

df["years_since_last_eval"] = pd.to_numeric(df["years_since_review"], errors="coerce")

df["confidence_level"] = (
    df["ConfidenceLevel"].astype(str).str.title().map(conf_map).fillna(1).astype(int)
)

df["risk_tier"] = (
    df["ReclassificationRiskTier"].astype(str).str.strip().str.title()
)

df = df[df["risk_tier"].isin({"Low", "Moderate", "High", "Critical"})]

df = df[
    df["ClinicalSignificance"]
    .astype(str)
    .str.contains(r"Pathogenic|Likely pathogenic", case=False, na=False)
]

out = ROOT / "outputs" / "risk_map_input.csv"
out.parent.mkdir(exist_ok=True)

cols = ["years_since_last_eval", "confidence_level", "risk_tier"]
df[cols].dropna().to_csv(out, index=False)

print("Saved:", out)


Loaded: (300000, 50)
['#AlleleID', 'Type', 'Name', 'GeneID', 'GeneSymbol', 'HGNC_ID', 'ClinicalSignificance', 'ClinSigSimple', 'LastEvaluated', 'RS# (dbSNP)', 'nsv/esv (dbVar)', 'RCVaccession', 'PhenotypeIDS', 'PhenotypeList', 'Origin', 'OriginSimple', 'Assembly', 'ChromosomeAccession', 'Chromosome', 'Start', 'Stop', 'ReferenceAllele', 'AlternateAllele', 'Cytogenetic', 'ReviewStatus', 'NumberSubmitters', 'Guidelines', 'TestedInGTR', 'OtherIDs', 'SubmitterCategories', 'VariationID', 'PositionVCF', 'ReferenceAlleleVCF', 'AlternateAlleleVCF', 'SomaticClinicalImpact', 'SomaticClinicalImpactLastEvaluated', 'ReviewStatusClinicalImpact', 'Oncogenicity', 'OncogenicityLastEvaluated', 'ReviewStatusOncogenicity', 'SCVsForAggregateGermlineClassification', 'SCVsForAggregateSomaticClinicalImpact', 'SCVsForAggregateOncogenicityClassification', 'ConfidenceLevel', 'LastEvaluated_dt', 'years_since_review', 'NumberSubmitters_num', 'conflicting', 'ReclassificationRiskScore', 'ReclassificationRiskTier']
Sa

In [3]:
mv "outputs/risk map input.csv" outputs/risk_map_input.csv


mv: rename outputs/risk map input.csv to outputs/risk_map_input.csv: No such file or directory
